In [40]:
%reset -f
import pandas as pd
import numpy as np
import mysql.connector
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel
df_user = pd.read_csv('Order_history.csv')
df_rest = pd.read_csv('Rest_Info.csv')

In [41]:
df_user.head()

,customer_id,order_id,rest_id
0,User9,1,6487f4c6fc13ae161d900937
1,User9,2,6487f4c6fc13ae161d900915
2,User8,3,6487f4c6fc13ae161d900939
3,User1,4,6487f4c6fc13ae161d900946
4,User1,5,6487f4c6fc13ae161d9008fc


In [42]:
df_rest.head()

,rest_id,rest_name,zone,rating,cuisine,budget
0,6487f4c6fc13ae161d9008e3,Durgan-Haag,North,1.4,mexican,5
1,6487f4c6fc13ae161d9008e4,Jacobson Group,South,2.6,indian,5
2,6487f4c6fc13ae161d9008e5,"Hintz, Harris and Beer",South,3.6,italian,1
3,6487f4c6fc13ae161d9008e6,Kling LLC,West,1.2,italian,3
4,6487f4c6fc13ae161d9008e7,Beer-Kuhic,North,4.7,italian,5


In [43]:
user = 'User7'
zone = 'South'

df_user = df_user[df_user['customer_id'] == user]

In [44]:
df = df_user.merge(df_rest, on  = "rest_id", how= "left").sort_values(by= "order_id")
df.head(10)

,customer_id,order_id,rest_id,rest_name,zone,rating,cuisine,budget
0,User7,6,6487f4c6fc13ae161d900945,Brakus and Sons,South,5.0,indian,5
1,User7,10,6487f4c6fc13ae161d900940,"Weimann, Gusikowski and Stracke",East,4.3,italian,3
2,User7,37,6487f4c6fc13ae161d900915,"Frami, McGlynn and VonRueden",North,1.8,indian,3
3,User7,41,6487f4c6fc13ae161d90093b,Tromp-Boehm,South,1.5,indian,3
4,User7,58,6487f4c6fc13ae161d90093c,"Grady, Purdy and Christiansen",West,3.5,indian,2
5,User7,68,6487f4c6fc13ae161d900941,"Sipes, Kshlerin and Kuhlman",West,4.2,indian,1
6,User7,70,6487f4c6fc13ae161d900936,Daugherty LLC,East,5.0,mexican,4
7,User7,72,6487f4c6fc13ae161d90093b,Tromp-Boehm,South,1.5,indian,3
8,User7,75,6487f4c6fc13ae161d900909,Mayert-Kunze,North,2.0,indian,5
9,User7,86,6487f4c6fc13ae161d9008e3,Durgan-Haag,North,1.4,mexican,5


In [54]:
# Replace the placeholders with your MySQL connection details
connection = mysql.connector.connect(
    host='localhost',
    port = '3306',
    user='root',
    password='sw23',
    database='flink'
)
cursor = connection.cursor()
query = "SELECT * FROM pending_orders_table;"
cursor.execute(query)
result = cursor.fetchall()

In [55]:
# convert SQL result to pandas dataframe and merge the restaurant info to the live restaurant data
rest_live_df = pd.DataFrame(result, columns= ["rest_id", "pending_count"])
del connection, cursor, query, result

In [57]:
# filter by zone -- 'hard coded' right now but get from POST request
rest_live_df = rest_live_df[rest_live_df['zone'] == zone]
rest_live_df = rest_live_df.merge(df_rest, on="rest_id", how = "left")


In [58]:
# one hot encoding categorical variables
def encode_categorical(df, columns):
    return pd.get_dummies(df, columns=columns)

In [59]:
user_orders_encoded = encode_categorical(df, ['cuisine'])
rest_live_encoded = encode_categorical(rest_live_df, ['cuisine'])

In [60]:
features = ['rating', 'budget', 'cuisine_indian', 'cuisine_italian', 'cuisine_mexican']

In [62]:
user_features = user_orders_encoded[features].values
rest_live_features = rest_live_encoded[features].values

In [63]:
similarity_matrix = cosine_similarity(user_features, rest_live_features)

In [77]:
similarity_df = pd.DataFrame(similarity_matrix, index=df.rest_id, columns=rest_live_df.index)

In [78]:
ranked_restaurants = similarity_df.idxmax(axis=1)